In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import cv2
import numpy as np
import os

In [ ]:
from invorec import common

In [ ]:
import time

In [ ]:
# dir_name = '/home/mobile/Documents/orientation_detection/data/test/inputs/vivo   1500/'
# dir_name = '/home/mobile/data/invoices/2017-04-21/inputs/'
# dir_name = '/home/mobile/data/invoice-edges/'
dir_name = '/home/mobile/data/invoices/2017-04-27-ord/'

In [ ]:
# im = cv2.imread('/home/mobile/data/invoice-edges/-64r134oif4m7pcMw8XHZjYlVxubL-czG6CVv9zBMGW4O1JL8H7GLg_nk1g8hUBy(2).jpg')
counter = 1
for filename in os.listdir(dir_name)[0:]:
    if os.path.splitext(filename)[1].lower() not in ['.jpg', '.jpeg']:
        print(filename)
        print(os.path.splitext(filename)[1].lower())
        continue
    print(counter)
    counter +=1
    path = os.path.join(dir_name, filename)
    im = cv2.imread(path)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

    width = 900
    ratio = float(width) / im.shape[1]
    im = cv2.resize(im, (width, int(im.shape[0]*ratio)))
    gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

    binary = cv2.adaptiveThreshold(
                    gray, 
                    255, 
                    cv2.ADAPTIVE_THRESH_MEAN_C,
                    cv2.THRESH_BINARY, 25, 8)
    binary = cv2.bitwise_not(binary)
    
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    tmp = cv2.cvtColor(binary, cv2.COLOR_GRAY2RGB)
    

    _, contours, _ = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        rect_ratio = float(max(w,h))/min(w,h)
        if (w < 30 or h < 30) or ( rect_ratio > 2):
            pass
        else:
            rect = x,y,w,h
            
            roi_image = common.clip_rect(tmp, rect)
            pos = np.where(roi_image)
            pos_roi = np.array([[px, py] for px,py in zip(pos[1], pos[0])])
            ellipse = cv2.fitEllipse(np.array(pos_roi))
            mask = np.zeros_like(roi_image)
            mask = cv2.ellipse(mask, ellipse, 255, 1)
            mask = mask.astype(np.float) / 255
            
            mask_count = np.sum(mask[np.where(mask)])
            image_mask_count = np.sum(roi_image[np.where(mask)]) / 255
            image_count = np.sum(roi_image) /255
#             print('mask count: {}'.format(mask_count))
#             print('image mask count: {}'.format(image_mask_count))
#             print('image count: {}'.format(image_count))
            
            prec = image_mask_count / mask_count
            recall = image_mask_count / image_count
            
            print('precision: {}'.format(prec))
            print('recall: {}'.format(recall))
            
            common.draw_rectangle_on_image(tmp, rect, (255, 0, 0), 2)
            
            if prec > 0.7:
                pos = np.array([[px+x, py+y] for px,py in zip(pos[1], pos[0])])
                ellipse = cv2.fitEllipse(np.array(pos))
                
                cv2.ellipse(tmp, ellipse, (0, 255, 0), 2)

    plt.figure(figsize=(16,16))
    plt.imshow(tmp)
    plt.show()
    time.sleep(1)

In [ ]:
count = 0
while count < im.shape[0] - 40:
    img_amount = cv2.cvtColor(im[count:count+50], cv2.COLOR_RGB2GRAY)
    img_amount = cv2.GaussianBlur(img_amount, (5, 5), 0)
    binary = cv2.Canny(img_amount, 5, 100, 0)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    lines = cv2.HoughLines(binary, 1, 1*np.pi/180, 600)
    
    if lines is not None and len(lines) > 0:
        for l in lines:
            rho,theta = l[0]
            a = np.cos(theta)
            b = np.sin(theta)+ 0.0001
            x0 = a*rho
            y0 = b*rho

            x1 = 0
            y1 = int(a/b * x0 + y0) + count

            x2 = binary.shape[1]
            y2 = int(-a/b * (x2 - x0) + y0) + count
            
            cv2.line(im, (x1,y1), (x2,y2), (255, 0, 0))
            
    count += 20

In [ ]:
plt.figure(figsize=(24, 16))
plt.imshow(im, 'gray')